# **equilibrium simulation true random**

In [14]:
import random
import pandas as pd

In [3]:
randomSeq_dict = {}
length_dna = 5000
for i in range(100): 
    chosen_trips =  random.choices(["A","T","C","G"],k=length_dna)
    dna = "".join(chosen_trips)
    randomSeq_dict["trial"+str(i)] = dna

## **running the simulation**

In [8]:
model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

In [4]:
#imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from numpy.random import choice
import copy 
#from tqdm import tqdm
import sys 
from datetime import datetime
import time
import timeit
import numpy as np 
import json
from tqdm import tqdm

In [5]:
#defining funtions ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def base_to_mutate(DNA, current_weights, indices): 
    """
    (DNA: str, weights: Dict) -> int
    
    Will return the index of a random choice of a base that will be mutated based on probabilities given by the
    weights dictionary (i.e. the model)
    ex. "ATCGTA" --> index 3 ("G") will mutate
    """

    # NORMALIZE THE POPULATION OF WEIGHTS    
    total_freq = sum(current_weights) - current_weights[0] - current_weights[-1] # remove the start and end weight
    normalized_weights = ["error"]*len(current_weights)
    for index, value in enumerate(current_weights):
        normalized_weights[index] = value/total_freq 
    
    normalized_weights[0] = 0
    normalized_weights[-1] = 0
    # DRAW THE INDEX OF THE BASE THAT WILL BE MUTATED
    base_index = choice(indices, p=normalized_weights) 
    

    # RETURN THE INDEX
    return base_index 

In [6]:
#triplets
triplets = []
for i_1 in ["A", "T", "G", "C"]: 
    for i_2 in ["A", "T", "G", "C"]: 
        for i_3 in ["A", "T", "G", "C"]: 
            triplets.append(i_1+i_2+i_3)

#parameters 
prop_muts = 2

In [9]:
for key,DNA in tqdm(randomSeq_dict.items()): 
# for DNA in chosen_intergen_randSeq: 
    
    trialN = list(randomSeq_dict.keys()).index(key)
    
    #write initial dna 
    file = open("data/trueRandom/{t}_DNAI.txt".format(t=trialN), "w")
    file.write(str(DNA))
    file.close()

    #set up ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #triplet-count dict 
    triplet_chosen_count_dict = {}
    triplet_into_count_dict = {}
    for triplet in triplets: 
        triplet_chosen_count_dict[triplet] = [0,0,0]  
        triplet_into_count_dict[triplet] = [0,0,0]

    #making the weights 
    current_weights = []
    for i in range(1,len(DNA)-1): 
        triplet = DNA[i-1:i+2]
        current_weights.append(model[triplet][0])

    indices = [i for i in range(1, len(DNA)-1)]
    mut_indices = []

    #making the av muts lists
    #creating the black plotting lists 
    av_mutability = []
    intron_mut = []
    exon_mut = []

    

    #runnign the sim ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    for i in range((int(len(DNA)*prop_muts))):
        base_index = base_to_mutate(DNA, current_weights, indices)

        mut_indices.append(base_index)

        #adding the count for "chosen to mutate" in coutns dict 
        c_triplet = DNA[base_index-1: base_index+2]
        c_triplet_left = DNA[base_index-2: base_index+1]
        c_triplet_right = DNA[base_index: base_index+3]    
        triplet_chosen_count_dict[c_triplet_left][0] += 1
        triplet_chosen_count_dict[c_triplet][1] += 1
        triplet_chosen_count_dict[c_triplet_right][2] += 1

        #accurate model INTO probability ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~OR ^ 
        curr_into_bases = []
        curr_into_bases_probs = []
        for into_base, prob in model[c_triplet][1].items(): 
            curr_into_bases.append(into_base)
            curr_into_bases_probs.append(prob)
        mb =  choice(curr_into_bases, p = curr_into_bases_probs)[1]


        #chanaging the dna 
        DNA = DNA[:base_index]+mb+DNA[base_index+1:]

        #updating the weights 
        m_triplet = c_triplet[0]+mb+c_triplet[2]
        m_triplet_left = c_triplet_left[0:2]+mb
        m_triplet_right = mb+c_triplet_right[1:3]
        current_weights[base_index-2]= model[m_triplet_left][0]    
        current_weights[base_index-1]= model[m_triplet][0]
        current_weights[base_index] = model[m_triplet_right][0]
        
        #adding the count for "mutated into thiss" in the triplet dict 
#         triplet_into_count_dict[m_triplet_left][0] += 1 
#         triplet_into_count_dict[m_triplet][1] += 1 
#         triplet_into_count_dict[m_triplet_right][2] += 1 

        #appending the av mut lists 
        av_mutability.append(np.mean(current_weights))


#     text_file = open("data/{t}_triplet_into_count_dict.txt".format(t=trialN), "w")
#     n = text_file.write(json.dumps(triplet_into_count_dict))
#     text_file.close()

#     text_file = open("data/{t}_triplet_chosen_count_dict.txt".format(t=trialN), "w")
#     n = text_file.write(json.dumps(triplet_chosen_count_dict))
#     text_file.close()

    file = open("data/trueRandom/{t}_DNA.txt".format(t=trialN), "w")
    file.write(str(DNA))
    file.close()

    file = open("data/trueRandom/{t}_avMut.txt".format(t=trialN), "w")
    file.write(json.dumps(av_mutability))
    file.close()


100%|██████████| 100/100 [27:37<00:00, 16.58s/it]


### **create plotting df**

In [11]:
len(data_dict["trial0"])

10000

In [12]:
data_dict = {"gens":[x for x in range(10000)]}
for i in tqdm(range(trialN)): 
    data_dict["trial"+str(i)] = json.load(open("data/trueRandom/{}_avMut.txt".format(i)))

100%|██████████| 99/99 [00:00<00:00, 208.78it/s]


In [15]:
data_df = pd.DataFrame.from_dict(data_dict)

In [16]:
data_df.to_csv("data/trueRandom/data_df_5kbp.csv")